# cuda-base

## 编译简化demo

In [2]:
import os 
class CudaAutoBuild(object):
    def __init__(self, root_dir = "/tmp/cuda-base"):
        self.root_dir = root_dir 
        assert root_dir.startswith("/tmp")
        os.makedirs(self.root_dir, exist_ok=True)
        os.system(f"rm -rf {os.path.join(self.root_dir, '*')}")


    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kws):
        pass 
    
    def add_file(self, filename, filestr):
        with open(os.path.join(self.root_dir, filename), "w") as f:
            f.write(f"{filestr.strip()}\n\n")

    def build(self, cmd=None):
        if cmd is None:
            cmd = "nvcc hello.cu --generate-code arch=compute_50,code=sm_50 -o hello"
        os.system(f"cd {self.root_dir} && {cmd}")
    
    def exec(self, cmd=None):
        if cmd is None:
            cmd = "hello"
        os.system(f"cd {self.root_dir} && chmod +x {cmd} && ./{cmd}")

In [21]:
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>

__global__ void print_HelloWorld(void) {
    printf("Hello World! from thread [%d, %d] From device.\n", threadIdx.x, blockIdx.x);
}

int main() {
    printf("Hello World from host!\n");
    print_HelloWorld<<<1, 1>>>();

    cudaDeviceSynchronize();
    return 0;
}
    """)
    cab.build()
    cab.exec()


Hello World from host!
Hello World! from thread [0, 0] From device.


## e01-HelloWorld

In [22]:
# create hello-world file
custr_HelloWorld = r"""
#include <stdio.h>

__global__ void print_HelloWorld(void) {
    printf("Hello World! from thread [%d, %d] From device.\n", threadIdx.x, blockIdx.x);
}

int main() {
    printf("Hello World from host!\n");
    print_HelloWorld<<<1, 1>>>();

    cudaDeviceSynchronize();
    return 0;
}
    """

!rm -rf build/*
!mkdir build
with open("build/hello-world.cu", "w") as f:
    f.write(custr_HelloWorld)


In [23]:
# build
!nvcc build/hello-world.cu \
    --generate-code arch=compute_50,code=sm_50 \
    -o build/hello-world


In [24]:
# exec
!chmod +x build/hello-world 
!build/hello-world

Hello World from host!
Hello World! from thread [0, 0] From device.


## e02-add

In [25]:
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>


// Definition of kernel functin to add two variable
__global__ void gpu_add(int d_a, int d_b, int *d_c) {
    *d_c = d_a + d_b;
}

// main function
int main() {
    // Defining host variable to store answer
    int h_a = 125, h_b = 236;
    int h_c;

    // Defining device pointer
    int *d_c;

    // Allocating memory for device pointer
    cudaMalloc((void**)&d_c, sizeof(int));

    // Kernal call
    gpu_add<<<1, 1>>>(h_a, h_b, d_c);

    // Copy result from device memory to host memory
    cudaMemcpy(&h_c, d_c, sizeof(int), cudaMemcpyDeviceToHost);

    printf("%d + %d = %d\n", h_a, h_b, h_c);

    // Free up memory
    cudaFree(d_c);

}
    """)
    cab.build()
    cab.exec()

125 + 236 = 361


## e02-device prop
- refer
  - https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html#structcudaDeviceProp

In [26]:
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>

// main function
int main() {
    
    int deviceCount = -1;
    cudaGetDeviceCount(&deviceCount);
    for (int device = 0; device < deviceCount; ++device) {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, device);
        printf("Device %d has compute capability %d.%d. \n",
            device, deviceProp.major, deviceProp.minor
        );
        printf("\t name: %s.\n", deviceProp.name);
        printf("\t maxThreadsDim: (%d, %d, %d).\n", deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1], deviceProp.maxThreadsDim[2]);
        printf("\t maxGridSize: (%d, %d, %d).\n", deviceProp.maxGridSize[0], deviceProp.maxGridSize[1], deviceProp.maxGridSize[2]);
        printf("\t maxThreadsPerBlock: %d.\n", deviceProp.maxThreadsPerBlock);
    }
    return 0;    

}
    """)
    cab.build()
    cab.exec()

Device 0 has compute capability 5.0. 
	 name: NVIDIA GeForce GTX 960M.
	 maxThreadsDim: (1024, 1024, 64).
	 maxGridSize: (2147483647, 65535, 65535).
	 maxThreadsPerBlock: 1024.


## add-mul
计算  
$y = a - \dfrac{b}{2} + c^2 $


In [27]:
# cuda 
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>

// Definition of kernel functin to add two variable
// y = a - b / 2 + c ** 2
__global__ void
arr_add(size_t n, float *d_a, float *d_b, float *d_c, float *d_y) {
    size_t tid = blockIdx.x;
    if (tid < n) {
        d_y[tid] = d_a[tid] - d_b[tid] / 2 + d_c[tid] * d_c[tid];
//        printf("idx a b c y: %05ld, %f, %f, %f, %f\n", tid, d_a[tid], d_b[tid],
//               d_c[tid], d_y[tid]);
    }
}


// main function
int main() {
    size_t n = 100000;
    float h_a[n], h_b[n], h_c[n], h_y[n];
    for (int i = 0; i < n; i++) {
        h_a[i] = (float) i;
        h_b[i] = (float) i;
        h_c[i] = (float) i;
        h_y[i] = 0;
    }

    float *d_a, *d_b, *d_c, *d_y;

    // Allocating memory
    cudaMalloc((void **) &d_a, n * sizeof(float));
    cudaMalloc((void **) &d_b, n * sizeof(float));
    cudaMalloc((void **) &d_c, n * sizeof(float));
    cudaMalloc((void **) &d_y, n * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_a, h_a, n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, n * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel call
    arr_add<<<n, 1>>>(n, d_a, d_b, d_c, d_y);

    // Copy result
    cudaMemcpy(h_y, d_y, n * sizeof(float), cudaMemcpyDeviceToHost);
    
    cudaDeviceSynchronize();

    float sum = 0;
    for (size_t i = 0; i < n; ++i) {
        sum += h_y[i];
    }
    // printf("sum: %f, ", sum);
    
    // Free
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_y);

    return 0;
}

    """)
    cab.build()
    cab.exec()

In [28]:
%%timeit  
!/tmp/cuda-base/hello 

189 ms ± 2.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
# cpp
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>

// Main Program
// y = a - b / 2 + c ** 2
void arr_add(size_t n, const float *a, const float *b, const float *c, float *y) {
    for (size_t i = 0; i < n; ++i) {
        y[i] = a[i] - b[i] / 2 + c[i] * c[i];
    }
}


int main() {
    size_t n = 100000;
    float a[n], b[n], c[n], y[n];
    for (int i = 0; i < n; i++) {
        a[i] = (float)i;
        b[i] = (float)i;
        c[i] = (float)i;
        y[i] = 0;
    }

    arr_add(n, a, b, c, y);
    float sum = 0;
    for (size_t i = 0; i < n; ++i) {
        sum += y[i];
    }
    // printf("sum: %f, ", sum);
}
    """)
    cab.build()
    cab.exec()

In [30]:
%%timeit  
!/tmp/cuda-base/hello 

123 ms ± 854 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### benchmark n 100000  
```txt
-------------------------------------------------------------
Benchmark                   Time             CPU   Iterations
-------------------------------------------------------------
bm_arr_add_cpu/1       351769 ns       351759 ns         1905
bm_arr_add_cpu/10     3320200 ns      3320090 ns          209
bm_arr_add_cpu/100   33445583 ns     33444053 ns           21


-------------------------------------------------------------
Benchmark                   Time             CPU   Iterations
-------------------------------------------------------------
bm_arr_add_gpu/1       502368 ns       502253 ns        10000
bm_arr_add_gpu/10     5023164 ns      5022271 ns         1000
bm_arr_add_gpu/100   50218850 ns     50204298 ns          100


----------------------------------------------------------------
Benchmark                      Time             CPU   Iterations
----------------------------------------------------------------
bm_arr_add_cpu/100         18050 ns        18046 ns        36805
bm_arr_add_cpu/1000      1690554 ns      1690532 ns          429
bm_arr_add_cpu/10000   177923426 ns    177912452 ns            4
bm_arr_add_cpu/100000 1.6662e+10 ns   1.6662e+10 ns            1

----------------------------------------------------------------
Benchmark                      Time             CPU   Iterations
----------------------------------------------------------------
bm_arr_add_gpu/100        287895 ns       287858 ns         2302
bm_arr_add_gpu/1000      5723494 ns      5723163 ns          138
bm_arr_add_gpu/10000   291581341 ns    291429314 ns            3
bm_arr_add_gpu/100000 2.7557e+10 ns   2.7556e+10 ns            1

```

## 向量点乘  - todo
计算两个向量的点乘， 点乘的公式如下，第1条公式是三维向量，第2条是n维向量
$$A*B = (x_1, x_2, x_3) * (y_1, y_2, y_3) = x_1 * y_1 + x_2 * y_2 + x_3 * y_3$$
$$\mathbf{a}*\mathbf{b} = \sum_{0}^{n}a_i * b_i$$


向量点乘用的那些方法：
1. 用到了cuda的__shared__变量的特性，在block块中，thread线程共享__shared__变量空间。
2. 用到了__syncthreads方法，用于块线程同步。该方法一般不放于if判断中，防止线程一直处于等待同步状态。
2. 用到了归约算法，


In [6]:
with CudaAutoBuild() as cab:
    cab.add_file("hello.cu", r"""
#include <stdio.h>

#define threadsPerBlock 8
#define N (threadsPerBlock * 3)


__global__ void vec_dot(const int *a, const int *b, int *c) {

    __shared__ int vec[threadsPerBlock];

    size_t idx = threadIdx.x + blockDim.x * blockIdx.x;
    size_t tid = threadIdx.x;

    if (idx<N) {
        vec[tid] = a[idx] * b[idx];
        printf("bid: %d, tid: %d, idx: %ld, \t%d * %d = %d, \n", blockIdx.x, threadIdx.x, idx, a[idx], b[idx], vec[tid]);
    }

    __syncthreads();

    if (tid == 0) {
        printf("vec: %d, %d, \n", vec[0], vec[threadsPerBlock-1]);
    }

    size_t size = threadsPerBlock / 2;

    while (size > 0) {

        if (tid < size) {
            vec[tid] = vec[tid] + vec[tid + size];
        }
        __syncthreads();
        size /= 2;
    }

    if (tid == 0) {
        c[blockIdx.x] = vec[0];
        printf("blockIdx.x=%d, ", blockIdx.x);
    }
}

int main() {

    size_t blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
    int h_a[N], h_b[N], h_c[blocksPerGrid];

    int *d_a, *d_b, *d_c;

    // assigned
    for (int i = 0; i < N; ++i) {
        h_a[i] = i;
        h_b[i] = 2;
    }

    // Malloc
    cudaMalloc((void **) &d_a, N * sizeof(int));
    cudaMalloc((void **) &d_b, N * sizeof(int));
    cudaMalloc((void **) &d_c, blocksPerGrid * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

    // Run kernel
    vec_dot<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c);

    // Copy data from device to host
    cudaMemcpy(h_c, d_c, blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost);

    cudaDeviceSynchronize();

    // sum
    printf("%d, ", h_c[0]);
    for (int i = 1; i < blocksPerGrid; ++i) {
        h_c[0] += h_c[i];
        printf("%d, ", h_c[i]);
    }
    printf("\n");
    printf("Result from gpu device: %d, \n", h_c[0]);
    printf("Result from cpu host: %d, \n", N * (N - 1));

    return 0;
}

    """)
    cab.build()
    cab.exec()



bid: 2, tid: 0, idx: 16, 	16 * 2 = 32, 
bid: 2, tid: 1, idx: 17, 	17 * 2 = 34, 
bid: 2, tid: 2, idx: 18, 	18 * 2 = 36, 
bid: 2, tid: 3, idx: 19, 	19 * 2 = 38, 
bid: 2, tid: 4, idx: 20, 	20 * 2 = 40, 
bid: 2, tid: 5, idx: 21, 	21 * 2 = 42, 
bid: 2, tid: 6, idx: 22, 	22 * 2 = 44, 
bid: 2, tid: 7, idx: 23, 	23 * 2 = 46, 
bid: 0, tid: 0, idx: 0, 	0 * 2 = 0, 
bid: 0, tid: 1, idx: 1, 	1 * 2 = 2, 
bid: 0, tid: 2, idx: 2, 	2 * 2 = 4, 
bid: 0, tid: 3, idx: 3, 	3 * 2 = 6, 
bid: 0, tid: 4, idx: 4, 	4 * 2 = 8, 
bid: 0, tid: 5, idx: 5, 	5 * 2 = 10, 
bid: 0, tid: 6, idx: 6, 	6 * 2 = 12, 
bid: 0, tid: 7, idx: 7, 	7 * 2 = 14, 
bid: 1, tid: 0, idx: 8, 	8 * 2 = 16, 
bid: 1, tid: 1, idx: 9, 	9 * 2 = 18, 
bid: 1, tid: 2, idx: 10, 	10 * 2 = 20, 
bid: 1, tid: 3, idx: 11, 	11 * 2 = 22, 
bid: 1, tid: 4, idx: 12, 	12 * 2 = 24, 
bid: 1, tid: 5, idx: 13, 	13 * 2 = 26, 
bid: 1, tid: 6, idx: 14, 	14 * 2 = 28, 
bid: 1, tid: 7, idx: 15, 	15 * 2 = 30, 
vec: 0, 14, 
vec: 32, 46, 
vec: 16, 30, 
blockIdx.x=0, blockIdx.x

## 矩阵乘法  --todo  

$$ 
M =
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$